In [2]:
import json
import os
import sys

In [3]:
from icecream import ic
import numpy as np
import pandas as pd
from pprint import pprint
import torch
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import densenet
from tqdm import tqdm

In [4]:
os.chdir('../')
sys.path.append('../dataset/')
import coco_data_prep
import coco_api_helper
import config_dataset

loading annotations into memory...
Done (t=12.58s)
creating index...
index created!
loading annotations into memory...
Done (t=5.36s)
creating index...
index created!
loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


In [5]:
annot_filepaths = {'train': config_dataset.train_annotation_filepath,
                 'validation' : config_dataset.valid_annotation_filepath,
                 'test': config_dataset.test_annotation_filepath}

with open(annot_filepaths['train'], 'r') as f:
    train_annot = json.load(f)
    
train_annot.keys()

dict_keys(['info', 'images', 'licenses', 'annotations', 'categories'])

### Create the Cateogries Label file

In [6]:
# cat_ids = [annot['id'] for annot in train_annot['categories']]
# cat_names = [annot['name'] for annot in train_annot['categories']]

# cat_table = pd.DataFrame(pd.Series(cat_ids, cat_names))
# cat_table.reset_index(drop=False, inplace=True)
# cat_table.rename(columns={
#     'index': 0,
#     0: 1
# }, inplace=True)
# cat_table = cat_table[[1, 0]]
# cat_table.to_csv('../dataset/coco_labels.txt', header=None)

In [7]:
coco_ann_trans = coco_data_prep.COCOAnnotationTransform()

### Helper Functions

In [8]:
def calc_new_dim(orig_h_or_w: int, padding, kernel, stride) -> int:
    """
    :orig_h_or_w: original width or original height of the input image
    given the original width or height, kernel size, pad width, stride size
    calculate the new width or height
    """
    new_dim = (orig_h_or_w + (2 * padding) - kernel) // stride + 1
    return new_dim

### Global Variables

In [9]:
with open('../dataset/categories.json', 'r') as j:
    categories = json.load(j)    

catid_to_catname_df = pd.DataFrame.from_records(pd.Series([c.values() for c in categories])).drop(0, axis=1)
catid_to_catname_df.to_csv('../dataset/coco_labels.txt', header=None, sep=',', mode='a')

In [10]:
train_np_data_dir = '../data/numpy_imgs/train_subset/'
train_jpg_data_dir = '../data/raw/train/train2014/'
train_annot_filepath = '../data/raw/train/annotations/instances_train2014.json'

with open('../dataset/imgs_by_supercategory.json', 'r') as f:
    desired_categories = json.load(f)

In [11]:
val_np_data_dir = '../data/numpy_imgs/val_subset/'
val_jpg_data_dir = '../data/raw/val/val2014/'
val_annot_filepath = '../data/raw/train/annotations/instances_val2014.json'

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')

### Load Data : Train & Validation into DataLoader

In [13]:
train_ds = coco_data_prep.COCODataset('train',
                                      train_np_data_dir, 
                                      train_annot_filepath,
                                      coco_data_prep.COCOAnnotationTransform(),
                                      sample_ratio=0.05,
                                      device='cpu')

loading annotations into memory...
Done (t=11.58s)
creating index...
index created!


100%|███████████████████████████████████████████| 82783/82783 [00:47<00:00, 1740.91it/s]


In [14]:
train_dl = coco_data_prep.get_dataloader(train_ds, 
                                         batch_size=512, 
                                         device='cpu')

In [15]:
val_ds = coco_data_prep.COCODataset('val',
                                    val_np_data_dir, 
                                    val_annot_filepath,
                                    coco_data_prep.COCOAnnotationTransform(),
                                    sample_ratio = 0.05,
                                    device='cpu')

loading annotations into memory...
Done (t=8.19s)
creating index...
index created!


100%|███████████████████████████████████████████| 40504/40504 [00:25<00:00, 1601.23it/s]


In [16]:
val_dl = coco_data_prep.get_dataloader(val_ds, 
                                     batch_size=512, 
                                     device='cpu')

### Load Model

#### Densenet121

In [17]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)

Using cache found in /home/ec2-user/.cache/torch/hub/pytorch_vision_v0.10.0


In [18]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

### Build From Scratch - Densenet Inspired

In [19]:
class DensenetInspired(nn.Module):
    def __init__(self):
        super(DensenetInspired, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu1 = nn.ReLU()
        self.mp1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        
        # This part is Densenet inspired...recycle use of BN, RELU, CONV repeatedly
        self.bn2 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu2 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        
        self.bn3 = nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu3= nn.ReLU(inplace=True)
        self.conv3 = nn.Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        
        self.flat = nn.Flatten()
        self.lin1 = nn.LazyLinear(out_features=1024)
        self.fc1 = nn.Linear(1024, 30)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.mp1(out)
        
        out = self.bn2(out)
        out = self.relu2(out)
        out = self.conv2(out)
        
        out = self.bn3(out)
        out = self.relu3(out)
        out = self.conv3(out)
        
        out = self.flat(out)
        out = self.lin1(out)
        out = self.fc1(out)

        return out

#### Modeling Variables

In [20]:
model = DensenetInspired()

/home/ec2-user/miniconda3/envs/cs7643-raml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:175: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [21]:
optim_params= dict(
    lr=0.001,
    betas=(0.9, 0.999),
    eps=1e-05,
    weight_decay=0.01,
    amsgrad=False,
)

optimizer = torch.optim.Adam(model.parameters(), **optim_params)

In [22]:
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(50):
    running_loss = 0.0
    count = 0
    
    for i, (batch) in tqdm(enumerate(train_dl)):
        image_batch, label_batch = [x[0] for x in batch], [x[1] for x in batch]
        image_batch = torch.stack(image_batch).to(torch.device('cpu:0')) 
        
        try:
            optimizer.zero_grad()
            outputs = model(image_batch)
            loss = criterion(outputs, label_batch.double())
            loss.backward()
            optimizer.step()
            count += 1
            running_loss += loss.item()
            #scheduler.step()
            if count % 100 == 99:    # print every 10 mini-batches
                print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, count + 1, running_loss / 100))
                running_loss = 0.0
        except:
            continue
#             print('not enough in batch_size')
    print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))

9it [00:26,  2.96s/it]

Epoch-0 lr: 0.001



9it [00:19,  2.13s/it]

Epoch-1 lr: 0.001



9it [00:18,  2.10s/it]

Epoch-2 lr: 0.001



3it [00:08,  2.48s/it]

In [ ]:
features_list = []
labels_list = []

# Use GPUs to speed up the inference, this should take around 10 minutes

model.to('cpu')
for batch in tqdm(train_dl):
    image_batch, label_batch = [x[0] for x in batch], [x[1] for x in batch]
    image_batch = torch.stack(image_batch).to('cpu') 

    with torch.no_grad():
        features_batch = model(image_batch) # .flatten(start_dim=1)
    features_list.append(features_batch)
    labels_list.extend(label_batch)

In [ ]:
type(train_ds[0][1])

In [ ]:
criterion(features_list[0], [44, 44, 59, 1, 44, 44, 79, 79])

In [ ]:
[[l['category_id'] for l in label] for label in labels_list]

In [ ]:
torch.cuda.empty_cache()

In [ ]:
features_list[0].shape

In [ ]:
for i, vecs in enumerate(features_list):
    torch.save(vecs, f'../data/torch_embeddings/densenet_pretrained_embs_len_50176-{i}.pt')

In [ ]:
(0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
(1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
(2): ReLU(inplace=True)
(3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)

In [ ]:
class DensenetInspired(nn.Module):
    def __init__(self):
        super(DensenetInspired).__init__()
        
        self.conv0 = model.features.conv0
        self.bn0 = model.features.norm0
        self.relu0 = model.features.relu0
        self.pool0 = model.features.pool0
        self.db0 = nn.BatchNorm2d(64, eps=1e-5, momentum=0.1, affine=True, track_running_stats=True)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(64, 32, kernel_size(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        self.linear = nn.Linear(in_features=1024, out_features=1000, bias=True)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x